#### Install memray 

You can use %package or %pip. 

This only needs to be run once per session, and only if memray is not in your Custom Environment.

In [ ]:
# %pip install memray xgboost plotly

#### Load 

Loading the extension makes the memray "magics" available in Jupyter

In [ ]:
%load_ext memray

#### Generate Data

In [ ]:
import bql
import xgboost as xgb
import plotly.express as px

In [ ]:
daterange = 29  # days
security = 'IBM US Equity'

basics_query = f"""get(
      px_last
    ) for(
      ['{security}']
    ) with(
      dates=range(-{daterange}d, 0d),
      fill=prev,
      currency=USD
    )"""

bql_svc = bql.Service()

response = bql_svc.execute(basics_query)
base_df = bql.combined_df(response)

# Reset the index: bql's combined_df returns ID as a sole index.
base_df = base_df.reset_index() 

base_df['date_ordinal'] = base_df['DATE'].apply(lambda x: x.toordinal())


#### memray_flamegraph

In [ ]:
%%memray_flamegraph

xg_df = base_df.copy()

xg_df['month'] = xg_df['DATE'].dt.month
xg_df['day_of_week'] = xg_df['DATE'].dt.dayofweek

train_df = xg_df.iloc[:-21]
test_df = xg_df.iloc[-21:]

X_train = train_df[['month', 'day_of_week']]
y_train = train_df["px_last"]

xmodel = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, objective='reg:squarederror')
xmodel.fit(X_train, y_train)

xg_df.loc[test_df.index, 'xgpredicted_px_last'] = xmodel.predict(test_df[['month', 'day_of_week']])

px.line(xg_df, x="DATE", y=["px_last", "xgpredicted_px_last"])


#### cprofile Profile

In [ ]:
%%prun 

xg_df = base_df.copy()

xg_df['month'] = xg_df['DATE'].dt.month
xg_df['day_of_week'] = xg_df['DATE'].dt.dayofweek

train_df = xg_df.iloc[:-21]
test_df = xg_df.iloc[-21:]

X_train = train_df[['month', 'day_of_week']]
y_train = train_df["px_last"]

xmodel = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, objective='reg:squarederror')
xmodel.fit(X_train, y_train)

xg_df.loc[test_df.index, 'xgpredicted_px_last'] = xmodel.predict(test_df[['month', 'day_of_week']])

px.line(xg_df, x="DATE", y=["px_last", "xgpredicted_px_last"])


In [ ]:
%pip install line_profiler
%load_ext line_profiler

def my_entrypoint():

    xg_df = base_df.copy()

    xg_df['month'] = xg_df['DATE'].dt.month
    xg_df['day_of_week'] = xg_df['DATE'].dt.dayofweek

    train_df = xg_df.iloc[:-21]
    test_df = xg_df.iloc[-21:]

    X_train = train_df[['month', 'day_of_week']]
    y_train = train_df["px_last"]

    xmodel = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, objective='reg:squarederror')
    xmodel.fit(X_train, y_train)

    xg_df.loc[test_df.index, 'xgpredicted_px_last'] = xmodel.predict(test_df[['month', 'day_of_week']])

    px.line(xg_df, x="DATE", y=["px_last", "xgpredicted_px_last"])


#### Use Line_Profiler %lprun

In [ ]:
%lprun -f my_entrypoint my_entrypoint()

#### Use timeit for Runtime

In [ ]:
%timeit my_entrypoint

#### Memory_profiler

In [ ]:
%pip install memory_profiler
%load_ext memory_profiler

In [ ]:
%memit my_entrypoint